In [1]:
import pandas as pd
import os
import numpy as np
pd.set_option('display.max_columns', 500)
# from data_preparation import prepare_triplogs, prepare_tripdatas, get_clean_triplogs, get_clean_tripdatas

In [2]:
!ls ../../../trips/may/source/triplogs
# !ls ../../../trips/2020_11_8_to_2020_11_14/CVP/tripdatas.csv
# !ls ../../../trips/2020_11_8_to_2020_11_14/CMS/opticomdevicelog.csv

05-01-2021 2021-04-30 2021-05-02


In [3]:
!pip install pyarrow

In [4]:
!ls /Users/ailin/Documents/trips/source/triplogs/*.parquet

/Users/ailin/Documents/trips/source/triplogs/part-00000-9f264ad0-dcee-4d1e-8bdd-633b9e271950.c000.snappy.parquet


In [ ]:
#  load data
triplogs = pd.read_csv('../../../trips/may/source/triplogs')
# tripdatas = pd.read_csv('../../../trips/2020_11_8_to_2020_11_14/CVP/tripdatas.csv')
# devicelog = pd.read_csv('../../../trips/2020_11_8_to_2020_11_14/CMS/opticomdevicelog.csv')

In [51]:
#  load data (parquet)
triplogs = pd.read_parquet('../../../trips/may/source/triplogs', engine='pyarrow')
tripdatas = pd.read_parquet('../../../trips/may/source/tripdatas', engine='pyarrow')
devicelog = pd.read_parquet('../../../trips/may/source/opticomdevicelog', engine='pyarrow')

In [35]:
list(triplogs.columns)

['__v',
 '_id',
 'countEarly',
 'countLate',
 'countOnTime',
 'countStopsHit',
 'countTspRequests',
 'deviceID',
 'direction',
 'duration',
 'earlyLimit',
 'endReason',
 'endStatus',
 'endTime',
 'headsign',
 'headway',
 'lateLimit',
 'logID',
 'routeName',
 'routePriority',
 'startTime',
 'stops',
 'syncedToCMS',
 'totalDwell',
 'totalTravel',
 'tripID',
 'tripType',
 'tspMode',
 'tzOffset',
 'uploadDate',
 'utcOffset',
 'valid',
 'vehicleName',
 'verASMApp',
 'verASMServer']

In [36]:
list(tripdatas.columns)

['__v',
 '_id',
 'deviceID',
 'dir',
 'distance',
 'dwellTime',
 'emitterID',
 'event',
 'headway',
 'loc',
 'logID',
 'mph',
 'routeName',
 'stopIndex',
 'stopName',
 'time',
 'trackTime',
 'travelTime',
 'unloc']

In [37]:
list(devicelog.columns)

['OpticomDeviceLogId',
 'DeviceId',
 'RegionName',
 'LocationName',
 'VehicleName',
 'VehicleId',
 'StartDateTime',
 'EndDateTime',
 'Duration',
 'ClassAndId',
 'Status',
 'FinalGreens',
 'FinalGreensTime',
 'MaxIntensity',
 'CreatedDateTime',
 'Class',
 'Id',
 'CausedPreempt',
 'Channel',
 'PriorityLevel',
 'AgencyName',
 'ChannelName',
 'DesiredGreenReceived',
 'AgencyRegionName',
 'Authorized',
 'ArchivedDate',
 'LocationId',
 'RegionId',
 'TurnSignalStatus',
 'NoPreemptCause',
 'AgencyID',
 'LogFormatType',
 'GPSClassId',
 'GPSClassDescription',
 'GreenAdjustment',
 'GreenAdjustmentTime',
 'GreenAdjustmentTimeSaved',
 'EntrySpeed',
 'AverageSpeed',
 'ExitSpeed',
 'RelativePriority',
 'DirectionalPriority',
 'TrackId',
 'InRangeDuration',
 'PreemptOutputUsed',
 'MinimumIntensity',
 'LogId',
 'IntersectionUnitIdType',
 'IntersectionUnitId',
 'Disabled',
 'EntryHeading',
 'EntryHeadingDirection',
 'ExitHeading',
 'ExitHeadingDirection',
 'EntryLatitude',
 'EntryLongitude',
 'ExitLatit

In [53]:
# create unique trip ID
triplogs['uid'] = triplogs['deviceID'] + "_" + triplogs['logID']
tripdatas['uid'] = tripdatas['deviceID'] + "_" + tripdatas['logID']

# convert to datetime and set time zone
triplogs['startTime'] = pd.to_datetime(triplogs['startTime']).dt.tz_localize('UTC')
triplogs['endTime'] = pd.to_datetime(triplogs['endTime']).dt.tz_localize('UTC')
tripdatas['time'] = pd.to_datetime(tripdatas['time']).dt.tz_localize('UTC')

devicelog['StartDateTime'] = pd.to_datetime(devicelog['StartDateTime']).dt.tz_localize('US/Pacific').dt.tz_convert('UTC')
devicelog['EndDateTime'] = pd.to_datetime(devicelog['EndDateTime']).dt.tz_localize('US/Pacific').dt.tz_convert('UTC')
devicelog['Duration'] = pd.to_timedelta(devicelog['Duration'], unit='second')
# create a col for real signal preemption
devicelog['event_start'] = devicelog['EndDateTime'] - devicelog['Duration']



In [39]:
start = '2021-05-2'
end = '2021-05-3'

In [40]:
# take only with TSP request
tripdatas = tripdatas[tripdatas['event'] == 'TSP request']
# join triplogs with tripdatas on 'uid'
logdatas = tripdatas.merge(triplogs, on='uid', how='inner')

# take only 1 day
mask = (logdatas['time'] > start) & (logdatas['time'] <= end)
logdatas = logdatas[mask]

mask = (devicelog['event_start'] > start) & (devicelog['event_start'] <= end)
devicelog = devicelog[mask]

In [41]:
devicelog.head()

,OpticomDeviceLogId,DeviceId,RegionName,LocationName,VehicleName,VehicleId,StartDateTime,EndDateTime,Duration,ClassAndId,Status,FinalGreens,FinalGreensTime,MaxIntensity,CreatedDateTime,Class,Id,CausedPreempt,Channel,PriorityLevel,AgencyName,ChannelName,DesiredGreenReceived,AgencyRegionName,Authorized,ArchivedDate,LocationId,RegionId,TurnSignalStatus,NoPreemptCause,AgencyID,LogFormatType,GPSClassId,GPSClassDescription,GreenAdjustment,GreenAdjustmentTime,GreenAdjustmentTimeSaved,EntrySpeed,AverageSpeed,ExitSpeed,RelativePriority,DirectionalPriority,TrackId,InRangeDuration,PreemptOutputUsed,MinimumIntensity,LogId,IntersectionUnitIdType,IntersectionUnitId,Disabled,EntryHeading,EntryHeadingDirection,ExitHeading,ExitHeadingDirection,EntryLatitude,EntryLongitude,ExitLatitude,ExitLongitude,PassengerCount,ConditionalPriority,MinutesLate,TransitVehicleId,TransitRoute,RouteRun,LocationGroupId,Green,FirstPhase,FirstPhaseState,FirstPhaseGT,FirstPhaseNonGT,SecondPhase,SecondPhaseState,SecondPhaseGT,SecondPhaseNonGT,event_start
3039,E2F7A1D1-08E2-4F31-9548-5F75EB47FE55,B62524CA-E16F-42FD-B149-F3D6124C8790,San Francisco,Market and 5th,NF8654,853D56FF-FD80-4048-A905-4F3016A10A58,2021-05-01 00:00:30+00:00,2021-05-02 00:01:46+00:00,0 days 00:00:00,10088654,-1,0,0,-1,2021-05-02 00:11:43.227,8,8654,False,0,1,MTA,EB & EBRT Market PH2/5,0,San Francisco,1,NaT,E5DA39D7-5C53-4691-93A3-850870094561,EA8E47B4-53CD-4BAA-90BF-3AE114D8C39A,0,1,10,1,AAA660AB-4A64-497C-BFF8-D82E652AD997,Regular Transit,0,0,0,0,0,0,-1,-1,14430289,0,0,-1,4028613,-1,,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,,,0,None,0,255,255,511,511,255,255,511,511,2021-05-02 00:01:46+00:00
4606,B7444A67-0BCD-4F96-96F7-92DC8B2B9CFD,633ACA5B-61CA-4860-B962-21CAF04A84F8,San Francisco,Geary and Park Presidio,NF6626,C416B951-1924-4934-B812-B885BA782021,2021-05-01 00:00:27+00:00,2021-05-02 00:01:48+00:00,0 days 00:00:07,10086626,-1,136,14,-1,2021-05-02 00:22:04.507,8,6626,True,3,1,MTA,WB Geary Ph8,0,San Francisco,1,NaT,682D6F11-E0F2-44D1-9D84-B485D9E5BB67,EA8E47B4-53CD-4BAA-90BF-3AE114D8C39A,0,128,10,1,AAA660AB-4A64-497C-BFF8-D82E652AD997,Regular Transit,0,0,0,35,6,0,-1,-1,10450291,13,4,-1,5453365,-1,,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,,,0,None,0,255,255,511,511,255,255,511,511,2021-05-02 00:01:41+00:00
6272,A9E1510E-4024-4C84-9DB2-C50BF3B8D108,354F1A56-A3CF-4786-B2CF-D000539A0E0A,San Francisco,Mission and 5th,NF7235,9F7E2C58-7CF0-4868-88B9-8C1C243537B8,2021-05-01 00:00:58+00:00,2021-05-02 00:01:23+00:00,0 days 00:00:00,10087235,-1,0,0,-1,2021-05-02 00:22:03.090,8,7235,False,2,1,MTA,WB Mission Phase 6,0,San Francisco,1,NaT,7FBE5244-5B7E-4B02-97DF-C3D130A7066F,EA8E47B4-53CD-4BAA-90BF-3AE114D8C39A,0,2,10,1,AAA660AB-4A64-497C-BFF8-D82E652AD997,Regular Transit,0,0,0,17,8,11,-1,-1,16378154,8,0,-1,5414443,-1,,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,,,0,None,0,255,255,511,511,255,255,511,511,2021-05-02 00:01:23+00:00
7321,869CEC57-150B-4014-9CB2-8A983FE92812,E4528C2D-9113-4429-A00A-6F99CF76DBA8,San Francisco,Mission and Allison,NF6682,E79F720D-2E85-4F03-A7F1-C1FD8897C94F,2021-05-01 00:00:56+00:00,2021-05-02 00:01:12+00:00,0 days 00:00:00,10086682,-1,136,19,-1,2021-05-02 00:11:36.273,8,6682,False,0,1,MTA,NB Mission Phase 2,0,San Francisco,1,NaT,5FD9947E-1482-4B89-B40C-2BBC43EB8DE7,EA8E47B4-53CD-4BAA-90BF-3AE114D8C39A,1,1,10,1,AAA660AB-4A64-497C-BFF8-D82E652AD997,Regular Transit,0,0,0,0,0,0,-1,-1,10147223,0,0,-1,1666126271,-1,,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,,,0,None,0,255,255,511,511,255,255,511,511,2021-05-02 00:01:12+00:00
16571,2FE78938-4EAE-4B15-B282-3C2A81199E5E,E9EA2CD2-2DA2-4545-884B-052FB5EDA3BA,San Francisco,Potrero and Midblock 22nd/23rd,NF5852,B68ABD42-CCD8-43D9-8CA4-F0C832239503,2021-05-01 00:01:00+00:00,2021-05-02 00:01:14+00:00,0 days 00:00:00,10085852,-1,0,0,-1,2021-05-02 00:21:59.830,8,5852,False,0,1,MTA,NB Potrero PH2,0,San Francisco,1,NaT,D5BD7A1B-ECAE-426A-8C37-0E1761E38F5E,EA8E47B4-53CD-4BAA-90BF-3AE114D8C39A,1,128,10,1,AAA660AB-4A64-497C-BFF8-D82E652AD997,Regular Transit,0,0,0,4,4,6,-1,-1,448228

In [42]:
# number of trips, which sent TSP requests
logdatas.uid.nunique()

1440

In [43]:
list(logdatas.columns)

['__v_x',
 '_id_x',
 'deviceID_x',
 'dir',
 'distance',
 'dwellTime',
 'emitterID',
 'event',
 'headway_x',
 'loc',
 'logID_x',
 'mph',
 'routeName_x',
 'stopIndex',
 'stopName',
 'time',
 'trackTime',
 'travelTime',
 'unloc',
 'uid',
 '__v_y',
 '_id_y',
 'countEarly',
 'countLate',
 'countOnTime',
 'countStopsHit',
 'countTspRequests',
 'deviceID_y',
 'direction',
 'duration',
 'earlyLimit',
 'endReason',
 'endStatus',
 'endTime',
 'headsign',
 'headway_y',
 'lateLimit',
 'logID_y',
 'routeName_y',
 'routePriority',
 'startTime',
 'stops',
 'syncedToCMS',
 'totalDwell',
 'totalTravel',
 'tripID',
 'tripType',
 'tspMode',
 'tzOffset',
 'uploadDate',
 'utcOffset',
 'valid',
 'vehicleName',
 'verASMApp',
 'verASMServer']

In [44]:
# merge opticomdevicelog and joint dataset of triplogs and tripdatas
devicelogdatas = logdatas.merge(devicelog, left_on='vehicleName', right_on='VehicleName', how='inner')

In [45]:
# take only trips in 15 minutes interval
devicelogdatas['timedelta'] = devicelogdatas.event_start - devicelogdatas.time
mask = (devicelogdatas.timedelta < pd.Timedelta("15 minutes")) & \
        (devicelogdatas.timedelta >= pd.Timedelta("0 minutes"))
filtered_df = devicelogdatas[mask]


In [46]:
filtered_df.uid.nunique()

1427

In [47]:
mask = (triplogs['startTime'] > start) & (triplogs['startTime'] <= end)
triplogs[mask].uid.nunique()

2208

In [48]:
trips = set(set(trips) - set(filtered_df.uid.unique()))

In [49]:
trips

{'4010KJ1113_20210430-t10',
 '4010KK2071_20210430-t22',
 '4010KK2071_20210430-t26',
 '4010KL2011_20210501-t3',
 '4010KL2011_20210501-t7',
 '4010KN2001_20210430-t1',
 '4010KN2001_20210430-t2',
 '4010KN2001_20210430-t3',
 '4010KN2001_20210430-t4',
 '4010KN2034_20210430-t3',
 '4010KN2034_20210430-t4',
 '4010KN2034_20210430-t5',
 '4010KN2034_20210430-t7',
 '4010KO2051_20210501-t3',
 '4010KO2056_20210501-t7',
 '4010KP2021_20210501-t8',
 '4010KS2010_20210430-t2',
 '4010KZ2006_20210501-t11'}

In [54]:
triplogs[triplogs['uid'].isin(trips)].to_csv('trips_did_request_no_in_opticom_v3.csv')

In [35]:
triplogs[triplogs['uid'].isin(trips)].shape

(22, 36)

In [20]:
trips =pd.read_csv('trips_did_request_no_in_opticom_v2.csv')

In [22]:
trips = trips.uid.unique()

In [50]:
len(trips)

18